<a href="https://colab.research.google.com/github/andentze/FaceColab_Unofficial/blob/main/FaceColab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Faceswap Notebook
 
Welcome to the Faceswap *not official* notebook. Ever had a problem with your PC/laptop not being good enough for Faceswap? This notebook will help you out with **everything** regarding Faceswap.
 
 
Feel free to set up the environment as you like. I, personally, prefer my dataset and the model being on the Google Drive directly, or something like this:
 
> "/content/drive/My Drive/colab/faceswap/model
 
> "/content/drive/My Drive/colab/faceswap/faces/A
 
> /content/drive/My Drive/colab/faceswap/faces/B
 
> /content/drive/My Drive/colab/faceswap/config
 
and so on.
 
You can choose to either use the GUI version of Faceswap(*which can take some time to set up*) or the CLI version of Faceswap that is present here.
 
You will also receive 4 kinds of GPUs: P4, T4, K80 or P100(*different kinds of GPU's in Colab Pro which are proven to be faster*). Also, you get 12 hours of notebook usage until you get disconnected(24 hours in Colab Pro). You *might* get frequent disconnects.
 
And before you start using this notebook, clone it to your Google Drive. (File -> Save a copy in Drive)

**I TAKE NO CREDIT FOR ANY REPOSITORIES, NOR SOFTWARE USED IN THIS NOTEBOOK. ANYTHING USED HERE IS NOT MINE.**


In [ ]:
#@title Check the current GPU
#@markdown Right here you can check what kind of GPU you have available right now. Run this code to output the GPU used.
#@markdown If it errors out, make sure your runtime type is set to "GPU".

!nvidia-smi

In [ ]:
#@title Keep-Alive Script
#@markdown You have to activate this to automatically reconnect if Colab disconnects you. Run this code, to activate the "Keep-Alive" script.
#@markdown And just to make sure, open the console using Ctrl+Shift+I, and paste the code below. This will work efficently for both CLI and DE.
 
import IPython
from google.colab import output
 
display(IPython.display.Javascript('''
function ClickConnect() {
  console.log('Working')
  document
    .querySelector('#top-toolbar > colab-connect-button')
    .shadowRoot.querySelector('#connect')
    .click()
}
 
setInterval(ClickConnect, 60000)
'''))
 
print("Done.")

> function ClickConnect() {

>  console.log('Working')

>  document

>    .querySelector('#top-toolbar > colab-connect-button')

>    .shadowRoot.querySelector('#connect')

>    .click()
}

>    setInterval(ClickConnect, 60000)

# Setting up the CLI

Not a fan of the GUI or it simply doesn't work? Look no furher. From this point you will install Faceswap directly to Colab without the need of the DE.

It was set up so even a toddler could figure out how it works.

In [ ]:
#@title Install Faceswap
#@markdown Install Faceswap using this code snippet.
#@markdown This will also copy your configuration files from your Drive, so specify the directory to each .ini file *if needed*.

#@markdown It will also ask for a Google authentication with a prompt. That is for a Google Drive mounting. You need this for persistance storage. Make sure your Drive is ready to use.
train = "train.ini dir(/content/dir/train.ini)" #@param {type:"string"}
extract = "extract.ini dir(/content/dir/extract.ini)" #@param {type:"string"}
convert = "extract.ini dir(/content/dir/convert.ini)" #@param {type:"string"}
#@markdown If you install Faceswap to your drive, you'll have to tick "drive_install" on every code chunk. But, it saves your time so you won't have to install Faceswap again everytime you crash.
install_to_drive = False #@param {type:"boolean"}

from IPython.display import clear_output
from google.colab import drive
drive.mount('/content/drive')

!git clone https://github.com/deepfakes/faceswap.git
!git clone https://github.com/andentze/facecolab_requirements
!cp "{train}" faceswap/config/
!cp "{extract}" faceswap/config/
!cp "{convert}" faceswap/config/
clear_output()
#!pip install -r faceswap/requirements_nvidia.txt
#!pip uninstall -y imageio
#!pip uninstall -y imageio-ffmpeg
!pip uninstall -y tensorflow
#!pip uninstall -y tensorflow-gpu

!pip install -r "/content/facecolab_requirements/requirements_nvidia.txt"
#!pip install "imageio>=2.9.0"
#!pip install "imageio-ffmpeg<0.4.3"
#!pip install "ffmpy==0.2.3"
#!pip install "tensorflow-gpu>=2.2.0,<2.5.0"

if install_to_drive:
  !cp "/content/faceswap" "/content/drive/My Drive/" -r
  print("Installed to Google Drive.")
  print("Deleting the old faceswap folder.")
  !rm -r "/content/faceswap/"
%env FACESWAP_BACKEND = "nvidia"
clear_output()
print("Done.")

And with that, you're good to go! You can go ahead, and use Faceswap at your own will!(*that is, until you hit a 12 hour mark.*)

**It appears Colab doesn't like long-terms calculations. So it will disconnect you at the most random times. BE AWARE OF THAT.**

# Workflow
 
Welcome to the Faceswap environment. Your first step is to extract the faces from your footage. After which you're going to have to fix your dataset. While PC/laptop users can fix that locally, mobile users will have to use the DE to do so.

In [ ]:
#@title Extraction
#@markdown Extraction is the first step for dataset creation. You **must** have good data in order for training to work better. Use this code snippet to extract your dataset.
 
#set variables start
 
input_dir = "/your/dir/to/videos/here" #@param {type:"string"}
output_dir = "your/dir/to/faces/here" #@param {type:"string"}
detector = "s3fd" #@param ["cv2-dnn", "mtcnn", "s3fd"]
alignment = "fan" #@param ["cv2dnn", "fan"]
masker = "unet-dfl" #@param ["bisenet-fp", "unet-dfl", "vgg-clear", "vgg-obstructed"]
al_normalization = "hist" #@param ["clahe", "hist", "mean"]
refeed = 3 #@param {type:"slider", min:0, max:10, step:1}
size = 512 #@param {type:"slider", min:256, max:2048, step:64}
extract_every_n = 1 #@param {type:"slider", min:1, max:30, step:1}
drive_install = False #@param {type:"boolean"}
if drive_install:
  path = "/content/drive/My Drive/faceswap/"
  print("The installation is on Google Drive. Using Drive directory.")
if not drive_install:
  path = "/content/faceswap/"
  print("The installation is local. Using local directory.")
#set variables end

!python3 '{path}'faceswap.py extract \
-i '{input_dir}' \
-o '{output_dir}' \
-D '{detector}' \
-A '{alignment}' \
-M '{masker}' \
-nm '{al_normalization}' \
-rf '{refeed}' \
-min 20 -l 0.4 \
-sz '{size}' \
-een '{extract_every_n}' \
-si 0 -ssf -L INFO

In [ ]:
#@title Extraction with saving faces
#@markdown Extraction is the first step for dataset creation. You **must** have good data in order for training to work better. Use this code snippet to extract your dataset.

#@markdown The script above extracts each face and does not save faces to the directory. Use this code to actually save faces and tinker around with them.

#set variables start

!python3 '{path}'faceswap.py extract \
-i '{input_dir}' \
-o '{output_dir}' \
-D '{detector}' \
-A '{alignment}' \
-M '{masker}' \
-nm '{al_normalization}' \
-rf '{refeed}' \
-min 20 -l 0.4 \
-sz '{size}' \
-een '{extract_every_n}' \
-si 0 -L INFO

In [ ]:
#@title Training
#@markdown Now, you have your dataset at tip-top shape! All you have to do is start training now! But wait. Before you actually start, make sure your configuration was set right.
 
#@markdown Made sure? Then get to training by setting up the variables first.
 
#set variables start
input_a = "your/dir/here/A" #@param {type:"string"}
input_b = "your/dir/here/B" #@param {type:"string"}
num_iterations = 1200000 #@param {type:"slider", min:100000, max:3000000, step:25000}
save_every = 360 #@param {type:"slider", min:360, max:1800, step:360}
save_model_every = 25000 #@param {type:"slider", min:25000, max:100000, step:25000}
batch_num = 48 #@param {type:"slider", min:4, max:72, step:4}
 
trainer_type = "dfaker" #@param ["lightweight", "original", "iae", "dfaker", "dlight", "unbalanced", "dfl-h128", "villain"]
 
model_dir = "/content/dir/your_model/" #@param {type:"string"}
timelapse_dir = "/content/dir/timelapse/" #@param {type:"string"}

drive_install = False #@param {type:"boolean"}
if drive_install:
  path = "/content/drive/My Drive/faceswap/"
  print("The installation is on Google Drive. Using Drive directory.")
if not drive_install:
  path = "/content/faceswap/"
  print("The installation is local. Using local directory.")
#set variables end
 
   #fit training args: -nw -nf
 
!python3 '{path}'/faceswap.py train \
  -A '{input_a}' \
  -B '{input_b}' \
  -m '{model_dir}' \
  -t '{trainer_type}' \
  -bs '{batch_num}' \
  -it '{num_iterations}' \
  -s '{save_every}' \
  -ss '{save_model_every}' \
  -tia '{input_a}' \
  -tib '{input_b}' \
  -to '{timelapse_dir}' \
  -w 

In [ ]:
#@title Training with No Warp
#@markdown Disabling the warp can let you achieve extra sharpness. Start this training sequence if you think your model has trained enough(*judge by the previews*).
#@markdown Your variables are saved from the training session above. If not, start your normal training sequence first, then terminate it.

drive_install = False #@param {type:"boolean"}
if drive_install:
  path = "/content/drive/My Drive/faceswap/"
  print("The installation is on Google Drive. Using Drive directory.")
if not drive_install:
  path = "/content/faceswap/"
  print("The installation is local. Using local directory.")

!python3 '{path}'/faceswap.py train \
  -A '{input_a}' \
  -B '{input_b}' \
  -m '{model_dir}' \
  -t '{trainer_type}' \
  -bs '{batch_num}' \
  -it '{num_iterations}' \
  -s '{save_every}' \
  -ss '{save_model_every}' \
  -tia '{input_a}' \
  -tib '{input_b}' \
  -to '{timelapse_dir}' \
  -w \
  -nw

In [ ]:
#@title Convert
#@markdown After your model is trained enough, you can convert your final result. Fill the variables and run the code yet another time.

#variables set
input_video = "/content/dir/videoA.mp4" #@param {type:"string"}
output_dir = "/content/dir/output" #@param {type:"string"}
alingments = "/content/dir/alignments.fsa" #@param {type:"string"}
model_dir = "/content/dir/your_model" #@param {type:"string"}
color_adj = "" #@param ["", "avg-color", "color-transfer", "manual-balance", "match-hist", "seamless-clone"]
mask = "" #@param ["", "components", "extended", "unet-dfl", "vgg-clear", "vgg-obstructed", "predicted"]
writer = "ffmpeg" #@param ["ffmpeg", "gif", "opencv", "pillow"]
output_scale = 100 #@param {type:"slider", min:25, max:400, step:5}
drive_install = False #@param {type:"boolean"}
if drive_install:
  path = "/content/drive/My Drive/faceswap/"
  print("The installation is on Google Drive. Using Drive directory.")
if not drive_install:
  path = "/content/faceswap/"
  print("The installation is local. Using local directory.")
#@markdown "Predicted" mask type is used only if you trained your own mask using "Learn Mask". If you're not sure, use "Components" or "Extended" as they are used by default.
#variables end

!python3 faceswap/faceswap.py convert \
-i '{input_video}' \
-o '{output_dir}' \
-al '{alignments}' \
-m '{model_dir}' \
-c '{color_adj}' \
-M '{mask}aa' \
-w '{writer}' \
-osc '{output_scale}' -l 0.4 -j 0 -L INFO

In [ ]:
#@title Convert with Swap Model
#@markdown Noticed that you trained the model the other way around? Do not worry. Run this to convert it the other way around.

#@markdown The variables should stay the same as from the code above.

!python3 faceswap/faceswap.py convert \
-i '{input_video}' \
-o '{output_dir}' \
-al '{alignments}' \
-m '{model_dir}' \
-c '{color_adj}' \
-M '{mask}aa' \
-w '{writer}' \
-osc '{output_scale}' -s -l 0.4 -j 0 -L INFO

# Various Tools

*Use if necessary.*

Faceswap provides a couple of handy tools for you to make your dataset managment easier. Here you can do whatever you want(*limited to the "Manual" tool as it requires a desktop environment*).

In [ ]:
#@title Download a video from YouTube

#@markdown Using this code snippet you can download your desired YouTube video to your Drive immediately. This is proven to save some time downloading it locally and copying it to the Drive.

#@markdown Go ahead and get your URL and the path with your desired quality and run the code.
from IPython.display import clear_output

!pip install pytube
clear_output()

from pytube import YouTube

url = "https://www.youtube.com/watch?v=yourlinkhere" #@param {type:"string"}
path = "/content/drive/MyDrive/your/dir/here" #@param {type:"string"}
video_name = "your_file_name.mp4" #@param {type:"string"}
#@markdown Make sure that your video supports your desired quality, else you're not going to succeed with the download.
quality = "1080p" #@param ["720p", "1080p"]

try:
  yt = YouTube(url)
  print("Downloading the video. Please wait.")
  yt.streams.filter(res = quality, progressive = True, file_extension = "mp4").first().download(output_path = path, filename = video_name)
  clear_output()
except:
  clear_output()
  print("Something went wrong. Please double-check your set variables, verify the video exists and try again.")
  print("Else use an external downloader and copy the file manually.")
print("Done.")


In [ ]:
#@title Faceswap Tools

#@markdown These are the most handy tools to use for managing the dataset using Faceswap.

In [ ]:
#@title Alignments
#@markdown This tool allows you to perform various actions to the alignments file. It is usually stored next to the video source file.
#@markdown You would only need "Remove Faces" job for most of the time.
#@markdown If you wish to use any other tool, proceed to the guides to find out how to use some of them.
#set variables
job = "remove-faces" #@param ["draw", "extract", "missing-alignments", "missing-frames", "multi-faces", "no-faces", "remove-faces", "rename", "sort"]
alignments = "/content/dir/alignments.fsa" #@param {type:"string"}
faces = "/content/dir/A or B" #@param {type:"string"}
drive_install = False #@param {type:"boolean"}
if drive_install:
  path = "/content/drive/My Drive/faceswap/"
  print("The installation is on Google Drive. Using Drive directory.")
if not drive_install:
  path = "/content/faceswap/"
  print("The installation is local. Using local directory.")
#end variables
!python3 faceswap\tools.py alignments \
-j '{job}' \
-o console \
-a '{alignments}' \
-fc '{faces}' \
-een 1 -sz 512 -L INFO


In [ ]:
#@title FFmpeg
#@markdown Generate videos for timelapse(*usually this tool is required for that*). **Strongly** recommended to use BRU first to rename each timestamp to start from 000000.jpeg.

#set variables
input = "your/timelapse/dir/here" #@param {type:"string"}
output = "your/video/output/here/example.mp4" #@param {type:"string"}
fps = 5 #@param {type:"slider", min:1, max:60}

#end variables
!python3 faceswap\tools.py effmpeg \
-a gen-vid \
-i input \
-o output \
-fps fps \
-ef .png \
-s 00:00:00 \
-e 00:00:00 \
-d 00:00:00 -sc 1920x1080 -L INFO

In [ ]:
#@title Sort

#@markdown This is the Sort tool. It helps you with cleaning your dataset more efficiently.

#@markdown The sorting may require **a lot** of RAM depending on your dataset quantity. The bigger your dataset is, the more RAM you may need.

#@markdown The formula for calculating the ammount of RAM is: *(n² * 24) / 1.8* where *n* is the number of images.
#@markdown The number you'll get is in bytes, so divide it by 1024 for megabytes, and by 1024 again to get gigabytes.

#@markdown If the sorting fails(*i.e. the OOM issue*), divide the dataset in some small parts and sort each one of them.

#@markdown Please note, that there are only those jobs, that **I** think are useful.
input = "/content/drive/MyDrive/colab_files/faceswap/faces/A" #@param{type:"string"}
job = "face" #@param ["face", "blur", "hist"]

!python3 faceswap/tools.py sort \
-i '{input}' \
-s '{job}' \
-t -1.0 -fp rename -g hist -b 5 -lf sort_log.json -L INFO

# Setting up the Desktop Environment
 
Now that we know we have a GPU available, we're set. Now, **you** choose what kind of environment you need or want.
 
*   Desktop Environment
*   Command Line(CLI)
 
A little side-note:
 
*   Desktop Environments take 5-8 minutes to set up, while CLI takes around 4 minutes.
*   Desktop Environments are a bit unstable, as Colab was not intended to use in such way. So, be careful.
 
The RDP snippet was taken from this repository:
https://github.com/PradyumnaKrishna/Colab-Hacks/

**YET TO BE IMPLEMENTED COMPLETELY, DO NOT USE UNTIL IT IS DONE. CLI IS USABLE FOR NOW**

**NOT RECOMMENDED TO USE**

In [ ]:
#@title Create a User
#@markdown Create a user here, you can leave the name and the password as default.
#@markdown This is **required** for the DE(Desktop Environment).
 
import os
 
username = "user" #@param {type:"string"}
password = "root" #@param {type:"string"}
 
print("Creating User and Setting it up")
 
# Creation of user
os.system(f"useradd -m {username}")
 
# Add user to sudo group
os.system(f"adduser {username} sudo")
    
# Set password of user to 'root'
os.system(f"echo '{username}:{password}' | sudo chpasswd")
 
# Change default shell from sh to bash
os.system("sed -i 's/\/bin\/sh/\/bin\/bash/g' /etc/passwd")
 
print("User Created and Configured")

In [ ]:
#@title Install the RDP
#@markdown Now, you can install RDP. Visit http://remotedesktop.google.com/headless and copy the command for Debian after authentication. Then, run this code. It will take 5-8 minutes to install.
 
import os
import subprocess
 
CRP = "DISPLAY= /opt/google/chrome-remote-desktop/start-host --code=\"4/0AY0e-g7_v-8XXFg429fOGy2bLQ_93HV7FdbGr7k6RtwqUEkzAoH6J0FH1MUmFZ3wyu7fNQ\" --redirect-url=\"https://remotedesktop.google.com/_/oauthredirect\" --name=$(hostname)" #@param {type:"string"}
 
#@markdown Enter a pin more or equal to 6 digits.
Pin = 123456 #@param {type: "integer"}
 
 
class CRD:
    def __init__(self):
        os.system("apt update")
        self.installCRD()
        self.installDesktopEnvironment()
        self.installGoogleChorme()
        self.finish()
 
    @staticmethod
    def installCRD():
        print("Installing Chrome Remote Desktop")
        subprocess.run(['wget', 'https://dl.google.com/linux/direct/chrome-remote-desktop_current_amd64.deb'], stdout=subprocess.PIPE)
        subprocess.run(['dpkg', '--install', 'chrome-remote-desktop_current_amd64.deb'], stdout=subprocess.PIPE)
        subprocess.run(['apt', 'install', '--assume-yes', '--fix-broken'], stdout=subprocess.PIPE)
 
    @staticmethod
    def installDesktopEnvironment():
        print("Installing Desktop Environment")
        os.system("export DEBIAN_FRONTEND=noninteractive")
        os.system("apt install --assume-yes xfce4 desktop-base xfce4-terminal")
        os.system("bash -c 'echo \"exec /etc/X11/Xsession /usr/bin/xfce4-session\" > /etc/chrome-remote-desktop-session'")
        os.system("apt remove --assume-yes gnome-terminal")
        os.system("apt install --assume-yes xscreensaver")
        os.system("systemctl disable lightdm.service")
 
    @staticmethod
    def installGoogleChorme():
        print("Installing Google Chrome")
        subprocess.run(["wget", "https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb"], stdout=subprocess.PIPE)
        subprocess.run(["dpkg", "--install", "google-chrome-stable_current_amd64.deb"], stdout=subprocess.PIPE)
        subprocess.run(['apt', 'install', '--assume-yes', '--fix-broken'], stdout=subprocess.PIPE)
 
    @staticmethod
    def finish():
        print("Finalizing")
        os.system(f"adduser {username} chrome-remote-desktop")
        command = f"{CRP} --pin={Pin}"
        os.system(f"su - {username} -c '{command}'")
        os.system("service chrome-remote-desktop start")
        print("Finished Succesfully")
 
 
try:
    if username:
        if CRP == "":
            print("Please enter authcode from the given link")
        elif len(str(Pin)) < 6:
            print("Enter a pin more or equal to 6 digits")
        else:
            CRD()
except NameError as e:
    print("username variable not found")
    print("Create a User First")

In [ ]:
#@title Mount the Google Drive
#@markdown Google Drive is used as Persistance HDD for files. It is mounted at your `user` folder. (*`root` if the username was not set*)
#@markdown This is for DE, look farther down for CLI.
 
def MountGDrive():
    from google.colab import drive
 
    ! runuser -l $user -c "yes | python3 -m pip install --user google-colab"  > /dev/null 2>&1
 
    mount = """from os import environ as env
from google.colab import drive
 
env['CLOUDSDK_CONFIG']  = '/content/.config'
drive.mount('{}')""".format(mountpoint)
 
    with open('/content/mount.py', 'w') as script:
        script.write(mount)
 
    ! runuser -l $user -c "python3 /content/mount.py"
 
try:
    if username:
        mountpoint = "/home/"+username+"/drive"
        user = username
except NameError:
    print("username variable not found, mounting at `/content/drive' using `root'")
    mountpoint = '/content/drive'
    user = 'root'
 
MountGDrive()

From there, you can install Faceswap *easily*. Or, you can use this to clean your dataset. But. Don't forget that Colab terminates your session if it's not active. So make sure your "Keep-Alive" script is active.

*After checking the installation, turns out it doesn't like a couple of dependicies.*

(yet to be checked and fixed if necessary)

**AGAIN, NOT RECOMMENDED TO USE**